# Получение структурированных данных

В этом разделе вы узнаете как можно извлекать структурированных данных с помощью GigaChat и вызова функций.

## Установка зависимостей

Для работы с примером установите зависимости:

In [1]:
!pip install langchain-gigachat python-dotenv -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Схема данных

Сначала нужно описать какую информацию требуется извлечь из текста.

Для описания примера схемы персональных данных используется Pydantic.

In [3]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Информация о человеке."""

    # Док-строка выше, передается в описании функции
    # и помогает улучшить результаты работы LLM

    # Обратите внимание:
    # * Все поля — необязательные (`optional`). Это позволяет модели не извлекать неописанные поля.
    # * У каждого поля есть описание (`description`), которое передается в модель, в описании аргументов функции.
    # Хорошее пописание помогает повысить качество извлечения.
    name: Optional[str] = Field(..., description="Имя человека")
    hair_color: Optional[str] = Field(
        ..., description="Цвет волос человека, заполни если известен"
    )
    height_in_meters: Optional[float] = Field(
        ..., description="Высота человека в метрах."
    )

При определении схем придерживайтесь следующих правил:

* Описывайте атрибуты и саму схему. Описания передаются в модель и используются для повышения качества извлечения информации.
* Не вынуждайте модель придумывать данные. В примере выше атрибуты отмечены как необязательные (`Optional`), что позволяет модели возвращать `None`, если она не знает ответа.

## Создание экстрактора

Пример ниже демонстрирует код экстрактора информации на основе заданной схемы.

In [4]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

# Определяем промпт: добавляем инструкции и дополнительный контекст
# На этом этапе можно:
# * Добавить примеры работы функций, для улучшения качества извлечения информации
# * Предоставить дополнительную информацию о том какие данные и откуда будут извлекаться
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ты эксперт в извлечении информации из текста. "
            "Извлекай только релевантную информацию из текста. "
            "Если ты не знаешь значение аттрибута, "
            "который нужно извлечь, поставь аттрибуту null.",
        ),
        # О том как повысить качество работы с помощью примеров
        # читайте в рекомендациях
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

## Инициализация GigaChat

Для работы с моделями GigaChat, нужно передать ключ авторизации GigaChat API.

Ключ можно указать в переменной среды `GIGACHAT_CREDENTIALS`, заданной в файле `.env` или созданной с помощью команды:

```sh
export GIGACHAT_CREDENTIALS=ключ_авториазации
```

При инициализации проверяется наличие переменной среды `GIGACHAT_CREDENTIALS` с заданным ключом авторизации GigaChat API.
Если переменная отсутствует, вы сможете указать ключ в поле **Введите ключ авторизации GigaChat API**.

О способах авторизации и поддерживаемых переменных среды — в [README библиотеки gigachat](https://github.com/ai-forever/gigachat).

In [ ]:
import getpass
import os
from dotenv import find_dotenv, load_dotenv
from langchain_gigachat.chat_models import GigaChat

load_dotenv(find_dotenv())

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Введите ключ авторизации GigaChat API: ")


llm = GigaChat(
    model="GigaChat-Pro",
    timeout=6000,
    top_p=0,
    verify_ssl_certs=False,
)

Создание и вызов цепочки.

In [8]:
runnable = prompt | llm.with_structured_output(schema=Person)

text = "Алан Смит блондин, 1.85 метра высотой"
runnable.invoke({"text": text})

Person(name='Алан Смит', hair_color='блондин', height_in_meters=1.85)

## Извлечение нескольких сущностей

В большинстве задача вам понадобится извлекать из текста не одну сущсноть, а список.

Для этого вы можете создавать вложенные модели с помощью pydantic.

In [10]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Информация о человеке."""

    # Док-строка выше, передается в описании функции
    # и помогает улучшить результаты работы LLM

    # Обратите внимание:
    # * Все поля — необязательные (`optional`). Это позволяет модели не извлекать неописанные поля.
    # * У каждого поля есть описание (`description`), которое передается в модель, в описании аргументов функции.
    # Хорошее пописание помогает повысить качество извлечения.
    name: Optional[str] = Field(..., description="Имя человека")
    hair_color: Optional[str] = Field(
        ..., description="Цвет волос человека, заполни если известен"
    )
    height_in_meters: Optional[float] = Field(
        ..., description="Высота человека в метрах."
    )


class Data(BaseModel):
    """Информация о людях."""

    # Создание модели, с помощью которой можно извлекать информацию о нескольких людях
    people: List[Person]

:::important

В этом разделе представлен общий пример извлечения информации, который может демонстрировать не самое высокое качество.

В разделе Руководства вы найдете информацию о том как повысить качество извлечения с помощью образцовых примеров.

:::

In [11]:
runnable = prompt | llm.with_structured_output(schema=Data)
text = (
    "Мое имя Джо, мои волосы черные и я 1.75 метра высотой. "
    "У Анны такие же волосы как у меня и она на 10 сантиметров меньше меня."
)
runnable.invoke({"text": text})

Data(people=[Person(name='Джо', hair_color='черные', height_in_meters=1.75), Person(name='Анна', hair_color='черные', height_in_meters=1.65)])

:::note

Когда схема подразумевает извлечение нескольких сущностей, она также позволяет модели не извлекать никакие сущности и возвращать пустой список, если в тексте нет подходящих данных.

Как правило это полезно, так как позволяет явно задать обязательные атрибуты сущности без необходимости вынуждать модель обнаруживать такую сущность.

:::